<a href="https://colab.research.google.com/github/sromanov103/Python-Chat-Bot/blob/main/%D0%91%D0%BE%D1%82_%D1%82%D0%B5%D0%BB%D0%B5%D0%B3%D1%80%D0%B0%D0%BC%D0%BC_%D0%BA%D0%B2%D0%B8%D0%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aiogram

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.5/513.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.2
    Uninstalling pydantic_core-2.16.2:
      Successfully uninstalled pydantic_core-2.16.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a

In [ ]:
!pip install aiosqlite

In [ ]:
# Установим библиотеку nest_asyncio
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

# Работа с ботом

In [ ]:
import aiosqlite
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram import F

# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)

# Замените "YOUR_BOT_TOKEN" на токен, который вы получили от BotFather
API_TOKEN = '6438544454:AAEZ-gBGMuFfcbezdUsSPNwRwEsBEOf5g6Y'

# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()

# Зададим имя базы данных
DB_NAME = 'quiz_bot.db'

# Добавим переменную для хранения очков пользователя
user_score = 0

async def create_table():
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect('quiz_bot.db') as db:
        # Создаем таблицу
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_state (user_id INTEGER PRIMARY KEY, question_index INTEGER)''')
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_results (user_id INTEGER PRIMARY KEY, score INTEGER, date TEXT)''')
        # Сохраняем изменения
        await db.commit()

async def create_table_results():
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect('quiz_bot.db') as db:
        # Создаем таблицу
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_results (user_id INTEGER PRIMARY KEY, score INTEGER, date TEXT)''')
        # Сохраняем изменения
        await db.commit()

# Структура квиза
quiz_data = [
    {
        'question': 'Что такое Python?',
        'options': ['Язык программирования', 'Тип данных', 'Музыкальный инструмент', 'Змея на английском'],
        'correct_option': 0
    },
    {
        'question': 'Какой тип данных используется для хранения целых чисел?',
        'options': ['int', 'float', 'str', 'natural'],
        'correct_option': 0
    },
    {
        'question': 'Какая операция используется для объединения двух строк в одну?',
        'options': ['+','-','*','/'],
        'correct_option': 0
    },
     {
        'question': 'Какой встроенный модуль используется для работы со строками?',
        'options': ['string','str','text', 'strings'],
        'correct_option': 2
    },
    {
        'question': 'Что такое асинхронное программирование?',
        'options': ['Функция','Метод построения опорных векторов','Выполнение ячеек кода','Параллельное выполнение нескольких задач'],
        'correct_option': 3
    },
    {
        'question': 'Какой из следующих операторов используется для цикла?',
        'options': ['for','while','do-while','repeat-until'],
        'correct_option': 0
    },
    {
        'question': 'Какой "оператор" используется для присваивания значения?',
        'options': [':=','==','=','=:'],
        'correct_option': 2
    },
    {
        'question': 'Какой метод используется для получения длины списка?',
        'options': ['lenght','size','count','len'],
        'correct_option': 3
    },
    {
        'question': 'Какой "оператор" используется для сравнения двух чисел?',
        'options': ['=','==','===','=:'],
        'correct_option': 1
    },
    {
        'question': 'Какой метод используется для добавления элемента в конец списка?',
        'options': ['append','add','push','insert'],
        'correct_option': 0
    },
]

def generate_options_keyboard(answer_options, right_answer):
    builder = InlineKeyboardBuilder()

    for option in answer_options:
        builder.add(types.InlineKeyboardButton(
            text=option,
            callback_data="right_answer" if option == right_answer else "wrong_answer")
        )

    builder.adjust(1)
    return builder.as_markup()


@dp.callback_query(F.data == "right_answer")
async def right_answer(callback: types.CallbackQuery):

    global user_score
    user_score += 1

    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    await callback.message.answer("Верно!")
    current_question_index = await get_quiz_index(callback.from_user.id)
    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)

    await save_quiz_result(callback.from_user.id, user_score)

    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer(f"Ваш итоговый счет: {user_score}")

# Сохранение результата в базу
async def save_quiz_result(user_id, score):
    async with aiosqlite.connect(DB_NAME) as db:
        try:
            # Вставляем новую запись
            await db.execute('INSERT INTO quiz_results (user_id, score, date) VALUES (?, ?, datetime("now"))', (user_id, score))
        except aiosqlite.IntegrityError:
            # Обновляем существующую запись
            await db.execute('UPDATE quiz_results SET score = ?, date = datetime("now") WHERE user_id = ?', (score, user_id))
        await db.commit()

@dp.callback_query(F.data == "wrong_answer")
async def wrong_answer(callback: types.CallbackQuery):
    global user_score

    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)
    correct_option = quiz_data[current_question_index]['correct_option']

    await callback.message.answer(f"Неправильно. Правильный ответ: {quiz_data[current_question_index]['options'][correct_option]}")

    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)

    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer(f"Ваш итоговый счет: {user_score}")


# Хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    builder = ReplyKeyboardBuilder()
    builder.add(types.KeyboardButton(text="Начать игру"))
    builder.add(types.KeyboardButton(text="Лидерборд"))
    await message.answer("Добро пожаловать в квиз!", reply_markup=builder.as_markup(resize_keyboard=True))


async def get_question(message, user_id):

    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(user_id)
    correct_index = quiz_data[current_question_index]['correct_option']
    opts = quiz_data[current_question_index]['options']
    kb = generate_options_keyboard(opts, opts[correct_index])
    await message.answer(f"{quiz_data[current_question_index]['question']}", reply_markup=kb)


# Начать новый квиз
async def new_quiz(message):
    global user_score
    user_score = 0
    user_id = message.from_user.id
    await reset_quiz_result(user_id)
    current_question_index = 0
    await update_quiz_index(user_id, current_question_index)
    await get_question(message, user_id)

# Сбросить результаты квиза для пользователя
async def reset_quiz_result(user_id):
    async with aiosqlite.connect(DB_NAME) as db:
        try:
            # Удаляем существующую запись
            await db.execute('DELETE FROM quiz_results WHERE user_id = ?', (user_id,))
        except aiosqlite.IntegrityError:
            pass
        await db.commit()


async def get_quiz_index(user_id):
     # Подключаемся к базе данных
     async with aiosqlite.connect(DB_NAME) as db:
        # Получаем запись для заданного пользователя
        async with db.execute('SELECT question_index FROM quiz_state WHERE user_id = (?)', (user_id, )) as cursor:
            # Возвращаем результат
            results = await cursor.fetchone()
            if results is not None:
                return results[0]
            else:
                return 0

#Вывод leaderboard
async def get_quiz_results():
    async with aiosqlite.connect('quiz_bot.db') as db:
        results = await db.execute('SELECT user_id, score, date FROM quiz_results ORDER BY score DESC')
        return await results.fetchall()

async def show_leaderboard(message: types.Message, results):
    if not results:
        await message.answer("Пока нет результатов.")
        return

    leaderboard = "\n".join([
        f"{i+1}. {(await bot.get_chat(user_id)).first_name} - {score} ({date})"
        for i, (user_id, score, date) in enumerate(results)
    ])

    await message.answer(f"Лидерборд:\n\n{leaderboard}")

@dp.message(lambda message: message.text == "Лидерборд")
async def cmd_leaderboard(message: types.Message):
    results = await get_quiz_results()
    await show_leaderboard(message, results)

async def update_quiz_index(user_id, index):
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Вставляем новую запись или заменяем ее, если с данным user_id уже существует
        await db.execute('INSERT OR REPLACE INTO quiz_state (user_id, question_index) VALUES (?, ?)', (user_id, index))
        # Сохраняем изменения
        await db.commit()


# Хэндлер на команду /quiz
@dp.message(F.text=="Начать игру")
@dp.message(Command("quiz"))
async def cmd_quiz(message: types.Message):

    await message.answer(f"Давайте начнем квиз!")
    user_score = 0
    await new_quiz(message)


# Запуск процесса поллинга новых апдейтов
async def main():

    # Запускаем создание таблиц базы данных
    await create_table()
    await create_table_results()

    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())